# target:
1. theta_bl truth level
   
2. theta_bl reco but truth matched
   
3. theta_bl truth in rest frame

In [6]:
import sys, os
sys.path.append(os.path.abspath("/afs/desy.de/user/z/zhuxinyu/public/mass_reco/"))
import numpy as np
import ROOT
from ROOT import TCanvas, TLegend, TFile, TPaveLabel, TPaveText, TTree, TLorentzVector, TH1D, TVectorT, gStyle, THStack, gPad, TH2D
from ROOT import gROOT
import helper
import math
import time
start_time = time.time()



## truth level theta_bl

In [7]:
file = TFile("../output.root")
tree = file.Get("nominal")

In [8]:
# 1D hists
theta_bl_truth = TH1D ("theta_bl_truth ","theta_bl_truth" ,50 ,-1 ,1)
theta_bl_reco = TH1D ("theta_bl_reco ","theta_bl_reco(truth match only)" ,50 ,-1 ,1)
theta_bl_minimax = TH1D ("theta_bl_minimax ","theta_bl_minimax" ,50 ,-1 ,1)
theta_t = TH1D ("theta_t ","cos #theta_t truth" ,50 ,-1 ,1)
theta_t_restframe = TH1D ("theta_t_restframe "," cos theta_t truth restframe" ,50 ,-1 ,1)

# 2D hists
theta_bl_2D = TH2D("theta_bl_2D", "theta_bl_reco" ,50 , -1 ,1, 50 ,-1 ,1)
theta_t_2D = TH2D("#theta_t_2D", "#theta_t boosted vs. restframe", 50 ,-1 ,1 , 50 , -1 ,1)
theta_t_vs_bl = TH2D("theta_t_vs_bl", "#theta_t  vs. bl minimax", 50 ,-1 ,1 , 50 , -1 ,1)
theta_reco_truth  = TH2D("theta_reco_truth", "theta_reco (mm bl)_truth(bl)", 50 ,-1 ,1 , 50 , -1 ,1)
theta_reco_truth_t  = TH2D("theta_reco_truth_t", "theta_reco (mm bl)_truth (t)", 50 ,-1 ,1 , 50 , -1 ,1)
theta_reco_truth_t_0_500  = TH2D("theta_reco_truth_t_0_500", "theta reco (mm bl)_truth (t), mttb < 500", 50 ,-1 ,1 , 50 , -1 ,1)
theta_reco_truth_t_500_800  = TH2D("theta_reco_truth_t_500_800", "theta_reco (mm bl)_truth (t) mtt = 500 ~ 800", 50 ,-1 ,1 , 50 , -1 ,1)
theta_reco_truth_t_800_inf  = TH2D("theta_reco_truth_t_800_inf", "theta_reco (mm bl)_truth (t) mtt > 800", 50 ,-1 ,1 , 50 , -1 ,1)

In [9]:
for n in range(tree.GetEntries()):
# for n in range(5000):
    if (not(n % 10000)): 
        print("event:", n, "--- %.2f seconds ---" % (time.time() - start_time))

    tree.GetEntry(n)
    ## 1. truth level llbb
    p4s_truth_b = []
    p4s_truth_b.append(helper.p4_from_pt_eta_phi_m(tree, "MC_b_from_t"))
    p4s_truth_b.append(helper.p4_from_pt_eta_phi_m(tree, "MC_b_from_tbar"))
    prefixes = ['MC_Wdecay1_from_t', 'MC_Wdecay2_from_t', 'MC_Wdecay1_from_tbar','MC_Wdecay2_from_tbar']
    p4s_truth_lep = helper.get_leptons_from_truth(prefixes, tree) # [lep from t, lep from tbar]

    ## 2. nominal level 
    p4s_lep = helper.GetLorenzP4List(tree, 'lep')
    p4s_bjet = helper.GetLorenzP4List(tree,'bjet')

    ## 3. truth match
    p4s_match_lep = helper.deltaR_match(p4s_truth_lep, p4s_lep, 0.4) # [lep from t, lep from tbar]
    p4s_match_bjet = helper.deltaR_match(p4s_truth_b, p4s_bjet, 0.4) # [bjet from t, bjet from tbar]

    ## if not 2b2l or match failed
    if len(p4s_match_lep)*len(p4s_match_bjet) == 0 : continue

    ## minimax
    if len(p4s_lep) !=2 or len(p4s_bjet) != 2: continue
    idx_l, idx_b = helper.minimax_cross(p4s_lep, p4s_bjet)
    

    ## make the variables
    bl_truth = [p4s_truth_b[0] + p4s_truth_lep[0], p4s_truth_b[1] + p4s_truth_lep[1]]
    bl_nom_match = [p4s_match_lep[0] + p4s_match_bjet[0], p4s_match_lep[1] + p4s_match_bjet[1]]
    bl_minimax = [p4s_lep[idx_l] + p4s_bjet[idx_b], p4s_lep[1 - idx_l] + p4s_bjet[1 - idx_b]]
    
    ################################ reboost #########################
    # 4vectors
    p4s_ttbar_beforeFSR = helper.p4_from_pt_eta_phi_m(tree, "MC_ttbar_beforeFSR")
    p4s_t_beforeFSR = helper.p4_from_pt_eta_phi_m(tree, "MC_t_beforeFSR")
    p4s_tbar_beforeFSR = helper.p4_from_pt_eta_phi_m(tree, "MC_tbar_beforeFSR")

    # boost vector
    BoostVector = p4s_ttbar_beforeFSR.BoostVector()

    # reboost every particle back
    p4s_t_beforeFSR_reboost = TLorentzVector(p4s_t_beforeFSR)
    p4s_t_beforeFSR_reboost.Boost(-BoostVector)

    p4s_tbar_beforeFSR_reboost = TLorentzVector(p4s_tbar_beforeFSR)
    p4s_tbar_beforeFSR_reboost.Boost(-BoostVector)

    ############################### fill ###################################
  
    
    
    for i in [0, 1]:
        theta_bl_truth.Fill(bl_truth[i].CosTheta())
        theta_bl_reco.Fill(bl_nom_match[i].CosTheta())
        theta_bl_minimax.Fill(bl_minimax[i].CosTheta())
        theta_bl_2D.Fill(bl_truth[i].CosTheta(), bl_nom_match[i].CosTheta())
        
    theta_t.Fill(p4s_t_beforeFSR.CosTheta())
    theta_t_restframe.Fill(p4s_t_beforeFSR_reboost.CosTheta())
    theta_t_2D.Fill(p4s_t_beforeFSR.CosTheta(), p4s_t_beforeFSR_reboost.CosTheta())

    theta_t.Fill(p4s_tbar_beforeFSR.CosTheta())
    theta_t_restframe.Fill(p4s_tbar_beforeFSR_reboost.CosTheta())
    theta_t_2D.Fill(p4s_tbar_beforeFSR.CosTheta(), p4s_tbar_beforeFSR_reboost.CosTheta())

    # theta t vs. bl minimax, fill the delta matched pair

    theta_t_vs_bl.Fill(p4s_t_beforeFSR_reboost.CosTheta(), bl_minimax[0].CosTheta())
    
    # theta_reco_truth: theta_bl_minimax_reco vs theta_bl_boosted(matched with reco_bl)
    matched_truth_bl = helper.deltaR_match(bl_minimax, bl_truth, 0.4)
    matched_truth_t = helper.deltaR_match(bl_minimax, [p4s_t_beforeFSR, p4s_tbar_beforeFSR], 0.8)
    if len(matched_truth_bl) * len(matched_truth_t) != 0:
        theta_reco_truth.Fill(bl_minimax[0].CosTheta(), matched_truth_bl[0].CosTheta())
        theta_reco_truth_t.Fill(bl_minimax[0].CosTheta(), matched_truth_t[0].CosTheta())
        theta_reco_truth.Fill(bl_minimax[1].CosTheta(), matched_truth_bl[1].CosTheta())
        theta_reco_truth_t.Fill(bl_minimax[1].CosTheta(), matched_truth_t[1].CosTheta())
        if p4s_ttbar_beforeFSR.M() < 500 * 1000:
            theta_reco_truth_t_0_500.Fill(bl_minimax[0].CosTheta(), matched_truth_t[0].CosTheta())
            theta_reco_truth_t_0_500.Fill(bl_minimax[1].CosTheta(), matched_truth_t[1].CosTheta())
        elif p4s_ttbar_beforeFSR.M() > 500 * 1000 and p4s_ttbar_beforeFSR.M() < 800 * 1000:
            theta_reco_truth_t_500_800.Fill(bl_minimax[0].CosTheta(), matched_truth_t[0].CosTheta())
            theta_reco_truth_t_500_800.Fill(bl_minimax[1].CosTheta(), matched_truth_t[1].CosTheta())
        elif p4s_ttbar_beforeFSR.M() > 800 * 1000:
            theta_reco_truth_t_800_inf.Fill(bl_minimax[0].CosTheta(), matched_truth_t[0].CosTheta())
            theta_reco_truth_t_800_inf.Fill(bl_minimax[1].CosTheta(), matched_truth_t[1].CosTheta())


event: 0 --- 0.47 seconds ---
event: 10000 --- 30.64 seconds ---
event: 20000 --- 60.68 seconds ---
event: 30000 --- 90.68 seconds ---
event: 40000 --- 120.83 seconds ---
event: 50000 --- 151.00 seconds ---
event: 60000 --- 181.18 seconds ---
event: 70000 --- 211.41 seconds ---
event: 80000 --- 241.77 seconds ---
event: 90000 --- 272.20 seconds ---
event: 100000 --- 302.68 seconds ---
event: 110000 --- 333.49 seconds ---
event: 120000 --- 363.96 seconds ---
event: 130000 --- 394.60 seconds ---
event: 140000 --- 424.88 seconds ---
event: 150000 --- 455.50 seconds ---
event: 160000 --- 486.27 seconds ---
event: 170000 --- 516.74 seconds ---
event: 180000 --- 547.36 seconds ---
event: 190000 --- 577.52 seconds ---
event: 200000 --- 607.49 seconds ---
event: 210000 --- 637.66 seconds ---
event: 220000 --- 667.77 seconds ---
event: 230000 --- 698.41 seconds ---
event: 240000 --- 728.57 seconds ---
event: 250000 --- 758.82 seconds ---


# save hists

theta_bl_truth

theta_bl_reco

theta_bl_2D


In [11]:
# fout = TFile("hists.root","recreate")
fout = TFile("temp_hists.root","recreate")

fout.cd()
theta_bl_truth.Write()
theta_bl_reco.Write()
theta_bl_minimax.Write()
theta_t.Write()
theta_t_restframe.Write()

theta_bl_2D.Write()
theta_t_2D.Write()
theta_t_vs_bl.Write()
theta_reco_truth.Write()
theta_reco_truth_t.Write()
theta_reco_truth_t_0_500.Write()
theta_reco_truth_t_500_800.Write()
theta_reco_truth_t_800_inf.Write()

fout.Write()
fout.Close()